In [1]:
import requests
import pandas as pd
import numpy as np
import random

In [2]:
!conda install -c conda-forge geopy --yes 

Solving environment: ...working... done

# All requested packages already installed.



In [3]:
from geopy.geocoders import Nominatim

In [4]:
from IPython.display import Image 
from IPython.core.display import HTML 

In [5]:
from pandas.io.json import json_normalize
import folium 

In [26]:
CLIENT_ID = '4T0ZTMF13HJ25D1HFGXC4RWF1USHSEUNLF1TFBXKVDF5T23M' # your Foursquare ID
CLIENT_SECRET = 'A04O0VP5BCWXBDGUO1A0N1OG4WHZQM3ZQZKUXXMPDGGBBMRS' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4T0ZTMF13HJ25D1HFGXC4RWF1USHSEUNLF1TFBXKVDF5T23M
CLIENT_SECRET:A04O0VP5BCWXBDGUO1A0N1OG4WHZQM3ZQZKUXXMPDGGBBMRS


In [27]:
newyork = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location_ny = geolocator.geocode(newyork)
latitude_ny = location_ny.latitude
longitude_ny = location_ny.longitude
print(latitude_ny, longitude_ny)

40.7149555 -74.0153365


In [28]:
toronto = '7 Charles St W, Toronto'

#geolocator = Nominatim(user_agent="foursquare_agent")
geolocator2 = Nominatim(user_agent="foursquare_agent")
location_tn = geolocator2.geocode(toronto)
#print (location_tn)

latitude_tn = location_tn.latitude
longitude_tn = location_tn.longitude
print(latitude_tn, longitude_tn)

43.6685152 -79.386693


In [29]:
print (location_tn)

7 West, 7, Charles Street West, Yorkville, University—Rosedale, Old Toronto, Toronto, Ontario, M4Y 2B7, Canada


In [30]:
search_query = 'Italian'
radius = 500
url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_ny, longitude_ny, VERSION, search_query, radius, LIMIT)
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_tn, longitude_tn, VERSION, search_query, radius, LIMIT)
print(url1)
print(url2)

https://api.foursquare.com/v2/venues/search?client_id=4T0ZTMF13HJ25D1HFGXC4RWF1USHSEUNLF1TFBXKVDF5T23M&client_secret=A04O0VP5BCWXBDGUO1A0N1OG4WHZQM3ZQZKUXXMPDGGBBMRS&ll=40.7149555,-74.0153365&v=20180604&query=Italian&radius=500&limit=100
https://api.foursquare.com/v2/venues/search?client_id=4T0ZTMF13HJ25D1HFGXC4RWF1USHSEUNLF1TFBXKVDF5T23M&client_secret=A04O0VP5BCWXBDGUO1A0N1OG4WHZQM3ZQZKUXXMPDGGBBMRS&ll=43.6685152,-79.386693&v=20180604&query=Italian&radius=500&limit=100


In [31]:
json1 = requests.get(url1).json()
json2 = requests.get(url2).json()
#print(json2)

In [32]:
venues_ny = json1['response']['venues']

dataframe_ny = json_normalize(venues_ny)
dataframe_ny.head(10)

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",294544,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/harrys-italian-p...,False,4fa862b3e4b0ebff2f749f06,225 Murray St,...,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,Harry's Italian Pizza Bar,v-1566253343
1,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f3232e219836c91c7bfde94,63 W Broadway,...,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71446, 'lng': ...",40.714460,-74.010086,10007,NY,Conca Cucina Italian Restaurant,v-1566253343
2,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,3fd66200f964a520f4e41ee3,124 Chambers St,...,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'label': 'display', 'lat': 40.71533713859952...",40.715337,-74.008848,10007,NY,Ecco,v-1566253343


In [33]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe_ny.columns if col.startswith('location_ny.')] + ['id']
dataframe_filtered = dataframe_ny.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,id
0,Harry's Italian Pizza Bar,Pizza Place,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,3fd66200f964a520f4e41ee3


In [36]:
dataframe_filtered.shape

(3, 3)